<a href="https://colab.research.google.com/github/Koks-creator/YoloKoalaDetection/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Dec 21 07:26:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

MessageError: ignored

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.14 MiB | 22.25 MiB/s, done.
Resolving deltas: 100% (10406/10406), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-12-21 07:28:14--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  39.9MB/s    in 5.8s    

2022-12-21 07:28:20 (26.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
 extracting: data/obj/classes.txt    
  inflating: data/obj/Image_1.jpg    
  inflating: data/obj/Image_1.txt    
  inflating: data/obj/Image_10.JPG   
  inflating: data/obj/Image_10.txt   
  inflating: data/obj/Image_100.jpg  
  inflating: data/obj/Image_100.txt  
  inflating: data/obj/Image_11.jpg   
  inflating: data/obj/Image_11.txt   
  inflating: data/obj/Image_12.jpg   
  inflating: data/obj/Image_12.txt   
  inflating: data/obj/Image_13.jpg   
  inflating: data/obj/Image_13.txt   
  inflating: data/obj/Image_15.jpg   
  inflating: data/obj/Image_15.txt   
  inflating: data/obj/Image_16.jpg   
  inflating: data/obj/Image_16.txt   
  inflating: data/obj/Image_17.jpg   
  inflating: data/obj/Image_17.txt   
  inflating: data/obj/Image_18.jpg   
  inflating: data/obj/Image_18.txt   
  inflating: data/obj/Image_19.jpg   
  inflating: data/obj/Image_19.txt   
  inflating: data/obj/Image_2.jpg    
  inflating: data/obj/Image_2.txt    
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.370122 0.510000 0.667073 0.787273

['0', '0.370122', '0.510000', '0.667073', '0.787273']
0 data/obj/Image_21.txt
0 0.370122 0.510000 0.667073 0.787273
0 0.363187 0.445051 0.627473 0.524745

['0', '0.363187', '0.445051', '0.627473', '0.524745']
1 data/obj/Image_48.txt
0 0.363187 0.445051 0.627473 0.524745
0 0.496094 0.518880 0.783203 0.891927

['0', '0.496094', '0.518880', '0.783203', '0.891927']
2 data/obj/Image_94.txt
0 0.496094 0.518880 0.783203 0.891927
0 0.508333 0.467460 0.326667 0.566667

['0', '0.508333', '0.467460', '0.326667', '0.566667']
3 data/obj/Image_50.txt
0 0.508333 0.467460 0.326667 0.566667
0 0.564583 0.495238 0.725833 0.984127

['0', '0.564583', '0.495238', '0.725833', '0.984127']
4 data/obj/Image_88.txt
0 0.564583 0.495238 0.725833 0.984127
0 0.484375 0.550529 0.523438 0.833137

['0', '0.484375', '0.550529', '0.523438', '0.833137']
5 data/obj/Image_47.txt
0 0.484375 0.550529 0.523438 0.833137
0 0.502421 0.488333 0.946731 0.933333

['0', '0.502421', '0.488333', '

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/Image_6.jpg', 'data/obj/Image_36.jpg', 'data/obj/Image_87.jpg', 'data/obj/Image_63.jpg', 'data/obj/Image_74.jpg', 'data/obj/Image_100.jpg', 'data/obj/Image_18.jpg', 'data/obj/Image_90.jpg', 'data/obj/Image_78.jpg', 'data/obj/Image_60.jpg', 'data/obj/Image_69.jpg', 'data/obj/Image_12.jpg', 'data/obj/Image_83.jpg', 'data/obj/Image_66.jpg', 'data/obj/Image_72.jpg', 'data/obj/Image_96.jpg', 'data/obj/Image_44.jpg', 'data/obj/Image_28.jpg', 'data/obj/Image_3.jpg', 'data/obj/Image_33.jpg', 'data/obj/Image_37.jpg', 'data/obj/Image_70.jpg', 'data/obj/Image_85.jpg', 'data/obj/Image_23.jpg', 'data/obj/Image_48.jpg', 'data/obj/Image_57.jpg', 'data/obj/Image_71.jpg', 'data/obj/Image_82.jpg', 'data/obj/Image_19.jpg', 'data/obj/Image_20.jpg', 'data/obj/Image_58.jpg', 'data/obj/Image_13.jpg', 'data/obj/Image_45.jpg', 'data/obj/Image_50.jpg', 'data/obj/Image_89.jpg', 'data/obj/Image_8.jpg', 'data/obj/Image_84.jpg', 'data/obj/Image_98.jpg', 'data/obj/Image_59.jpg', 'data/obj/Image_32.jpg', '

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 141539, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.916275), count: 4, class_loss = 0.000000, iou_loss = 0.024413, total_loss = 0.024414 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 141543, rewritten_bbox = 0.0